In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

np.random.seed(0)


: 

# Predictive Linear Regression

Brandon Martin-Anderson June 2019, with great debt to Moses Marsh, Matt Drury

### Learning goals

- Describe the essential aspects of a prediction task.
- Why estimate?
- How do we test a model?
- Fit a linear model and evaluate its performance with and without `sklearn`.
- Use multiple features, categorical features, and engineered features.
- Pick polynomial regression order using cross-validation.

### The task of predicting quantities: in story form

Let's start a business: **we estimate the fuel economy of cars.** People describe a car and pay us money, and we tell them the fuel economy (in miles per gallon) of their car.

- The first customer comes in, and says, "This is a very competitive field. Why should we pay you?"
- You say, "We are very good at what we do."
- They say, "Your competitor says the same thing. Do you have evidence?"

Note two things.

1. **You are competing against other models.**
2. **Your model will be evaluated quantitatively.**

So you say: "We have analyzed all of our competitors's predictions. All their predictions have some error; the mean of their predictive errors is 5 mpg. We can do better."

- Them: "Ah, interesting. Prove it."
- You: "Describe a car."
- Them: "A 2020 hybrid Ford F250"
- You: "Can't predict - that **that car isn't in our records**"
- Them: "What good is that?"
- You: "If you ask about an old car, we can predict it perfectly."
- Them: "And...why would we pay for that?"
- You: "I don't know, but we are perfect."
- Them: "We'll go to your competitor instead."

3. **The only performance that matters is predictive performance on unseen examples.**


## A very simple predictive model

We wish to devise a predictive model $\hat{f}$ for our fuel-economy-estimation business. We start with a CSV we found on the internet.


In [ ]:
cars = pd.read_csv("data/cars.csv", na_values=["?"])


In [ ]:
cars = cars.dropna()


In [ ]:
cars.head()


Let's start with a very simple model: **always predict the average mpg**.


## Evaluating our model

Let's evaluate our model.

Because our metric for success is performance on unseen data, we need to keep some our data unseen.


In [ ]:
n = len(cars)
n_holdout = int(n*0.2)
print(f"{n} records total, holding out {n_holdout}")


Shuffle the cars using [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html); or else we'd learn just about the 8-cylinder cars (they're ordered in the dataset)


In [ ]:
cars = cars.sample(len(cars))
cars_test = cars.iloc[:n_holdout]
cars_train = cars.iloc[n_holdout:]


In [ ]:
cars_train.head()


Creating the "model" simply involves finding the mean, and returning it for every row in $X$.


In [ ]:
mean_mpg = cars_train.mpg.mean()


In [ ]:
mean_mpg


In [ ]:
def fhat(X): return np.ones(len(X))*mean_mpg  # our model!


In [ ]:
yhat = fhat(cars_test)  # predictions for all cars!


In [ ]:
yhat


Find the MSE without loops using vectorized numpy-style operations:


In [ ]:
mse = ((cars_test.mpg - yhat)**2).mean()
mse


A rough estimate, but it satisfies two important requirements:

- It is a quantitative, comparable error metric,
- computed on unseen data.

In a very simple way, we've done our job. Simple models like this are called **benchmarks**; they're very useful for setting a lower bound on the performance of more complex models. **This is our number to beat**.


Though MSE is mathematically elegant, it can be difficult to interpret.
It is also common to look at the square root of the MSE (RMSE), which has
the same units as the target variable.


In [ ]:
mse**0.5


This can be interpreted as how far off the model is on average. So on average (roughly, 1 standard deviation or less away), our estimate of 24 mpg is off from the actual mpg in our test data by 8 mpg.


## Linear regression as linear algebra

The mpg (y) can be modeled as a function of the car weight (X)

$$\hat{y_i} = mx_i + b$$

We can express this as

$$\hat{y} = X\beta$$

where

- $\hat{y_i}$ is the ith item in $\hat{y}$
- $x_i$ is the ith row in $X$
- $\beta$ is $(m,b)$

This is a small notational convenience that will become a huge convenience soon.


In [ ]:
X = np.stack([cars_train.weight.values, np.ones_like(cars_train.weight)]).T
y = cars_train.mpg.values


In [ ]:
beta = np.linalg.inv(X.T @ X) @ X.T @ y  # was not aware of @ as dot product
beta


In [ ]:
m, b = beta  # slope, intercept
def fhat(X): return m*X + b


In [ ]:
fig, ax = plt.subplots(figsize=(9, 7))
ax.scatter(cars_test.weight, cars_test.mpg, label="data")
ax.set_xlabel("weight (lbs)")
ax.set_ylabel("fuel economy (mpg)")
ax.set_title("test set fuel economy data & predictive model")

weights = np.linspace(1500, 5000)
ax.plot(weights, fhat(weights), c="red", label="model")
ax.legend()

for x_i, y_i in zip(cars_test.weight, cars_test.mpg):
    plt.plot([x_i, x_i], [y_i, m*x_i+b], color='gray', linestyle='dashed')


In [ ]:
yhat = fhat(cars_test.weight)
mse = ((cars_test.mpg - yhat)**2).mean()
print(mse)
rmse = np.sqrt(mse)
print(rmse)


We've halved our RMSE, a stunning success using linear regression!


## Linear regression on multiple predictors

As a notational convenience we coined:

$$\hat{y} = X\beta$$

We can generalize $X$ to a n by p+1 matrix, where row i corresponds to the ith sample, and column j corresponds to the jth predictor. Likewise $\beta$ is a vector of length p+1.

If we crack this open, it expands to:

$$ \hat{y*i} = \beta_0 \cdot 1 + \beta_1x*{i,1} + \beta*2x*{i,2} + ... + \beta*px*{i,p} $$

For example,

- 0th column would be all 1s,
- the 1st column of X holds the car weight,
- the 2st column of X holds the cylinder displacement.

In this case the semantics of $\beta$ is:

- $\beta_0$ is the "base" car mpg
- $\beta_1$ is the mpg penalty for each pound of car
- $\beta_2$ is the mpg penalty for each cc of engine displacement

Let's trust the math and try it:


In [ ]:
X = np.stack([np.ones_like(cars_train.weight),
              cars_train.weight.values,
              cars_train.displacement.values]).T
y = cars_train.mpg.values


In [ ]:
X[:10]


In [ ]:
beta = np.linalg.inv(X.T @ X) @ X.T @ y  # the closed-form OLS solution
beta


In [ ]:
X_test = np.stack([np.ones_like(cars_test.weight),
                   cars_test.weight.values,
                   cars_test.displacement.values]).T
y_test = cars_test.mpg.values


In [ ]:
mse = ((X_test @ beta - y_test)**2).mean()
rmse = np.sqrt(mse)
print(mse)
print(rmse)


Adding displacement improves performance yet again.


## Linear regression using categorical predictors

In the cars dataset, the feature `origin` corresponds to the _continent_ or origin.


In [ ]:
cars.origin.value_counts()


This looks like a numerical feature, but it isn't at all - the codes correspond to the USA, Europe, and Asia in increasing order. How do we use the tools of linear regression on a feature that's _not a number_?

One simple strategy is to create numerical "dummy variables" that encode the presence of a categorical feature with a 1.0.


In [ ]:
cars["american"] = cars.origin.map(lambda x: 1.0 if x == 1 else 0.0)
cars["european"] = cars.origin.map(lambda x: 1.0 if x == 2 else 0.0)
cars["asian"] = cars.origin.map(lambda x: 1.0 if x == 3 else 0.0)


In [ ]:
cars.head()


Now there exists a linear relationship between the category-dummy and the
target variable. For example as a car's "american" property goes to 1.0,
the fuel economy goes down:


In [ ]:
plt.scatter(cars.american, cars.mpg)
plt.xlabel("American-ness")
plt.ylabel("fuel economy (mpg)")


In [ ]:
# note only using "American or not" in model below
cars_test = cars.iloc[:n_holdout]
cars_train = cars.iloc[n_holdout:]

X = np.stack([np.ones_like(cars_train.weight),
              cars_train.weight.values,
              cars_train.displacement.values,
              cars_train.american.values]).T
y = cars_train.mpg.values

beta = np.linalg.inv(X.T @ X) @ X.T @ y

X_test = np.stack([np.ones_like(cars_test.weight),
                   cars_test.weight.values,
                   cars_test.displacement.values,
                   cars_test.american.values]).T
y_test = cars_test.mpg.values

mse = ((X_test @ beta - y_test)**2).mean()
print(mse)
rmse = np.sqrt(mse)
print(rmse)


Once again we were able to squeak out a small improvement on the test set.


## Engineered features: making curves out of straight lines

The relationship between weight and fuel economy is _not a straight line_. This makes sense if you think about it: the fuel economy of a car can never be negative - it will go to zero as the weight increases.


In [ ]:
fig, ax = plt.subplots(figsize=(9, 7))
ax.scatter(cars.weight, cars.mpg)
ax.set_xlabel("weight (lbs)")
ax.set_ylabel("fuel economy (mpg)")
ax.set_title("car weight vs. fuel economy")


One solution is to perform linear regression on engineered features. One potential model:

$$y = \beta_0 + \frac{\beta_1}{x_{weight}}$$

might seem nonlinear, but it's simply a linear function of the predictor $\frac{1}{x_{weight}}$.


In [ ]:
X = np.stack([np.ones_like(cars_train.weight),
              1/cars_train.weight.values]).T
y = cars_train.mpg.values

beta = np.linalg.inv(X.T @ X) @ X.T @ y
beta

fig, ax = plt.subplots(figsize=(9, 7))
ax.scatter(cars.weight, cars.mpg)
weightspace = np.linspace(1500, 5000)
ax.plot(weightspace, beta[1]*(1/weightspace)+beta[0], c="red")
ax.set_xlabel("weight (lbs)")
ax.set_ylabel("fuel economy (mpg)")
ax.set_title("car weight vs. fuel economy")


In [ ]:
X_test = np.stack([np.ones_like(cars_test.weight),
                   1/cars_test.weight.values]).T
y_test = cars_test.mpg.values


mse = ((X_test @ beta - y_test)**2).mean()
print(mse)
rmse = np.sqrt(mse)
print(rmse)


One popular strategy for fitting complex curves is **polynomial regression**, which involves creating polynomial features from observed features.


In [ ]:
# going up to cubic features

X = np.stack([np.ones_like(cars_train.weight),
              cars_train.weight.values**1,
              cars_train.weight.values**2,
              cars_train.weight.values**3]).T
y = cars_train.mpg.values

beta = np.linalg.inv(X.T @ X) @ X.T @ y

fig, ax = plt.subplots(figsize=(9, 7))
ax.scatter(cars.weight, cars.mpg)
weightspace = np.linspace(1500, 5000)
fake_feats = np.stack([np.ones_like(weightspace),
                       weightspace**1,
                       weightspace**2,
                       weightspace**3]).T
ax.plot(weightspace, fake_feats @ beta, c="red")
ax.set_xlabel("weight (lbs)")
ax.set_ylabel("fuel economy (mpg)")
ax.set_title("car weight vs. fuel economy")


#### Question: which do you think is a preferable featurization in this case? 1/weight or polynomial approach?


## Cross-validation for hyperparameter selection


It is possible to go overboard with polynomial features, leading to poor generalization on the test set:


In [ ]:
order = 3  # try 3, 5, 8, 10

features = [np.ones_like(cars_train.weight)]
for i in range(1, order+1):
    features.append(cars_train.weight.values**i)
X = np.stack(features).T
y = cars_train.mpg.values

beta = np.linalg.inv(X.T @ X) @ X.T @ y

fig, ax = plt.subplots(figsize=(9, 7))
ax.scatter(cars.weight, cars.mpg)
weightspace = np.linspace(1500, 5000)
fake_feats = [np.ones_like(weightspace)]
for i in range(1, order+1):
    fake_feats.append(weightspace**i)
fake_feats = np.stack(fake_feats).T
ax.plot(weightspace, fake_feats @ beta, c="red")
ax.set_xlabel("weight (lbs)")
ax.set_ylabel("fuel economy (mpg)")
ax.set_title("car weight vs. fuel economy")


In [ ]:
orders = range(1, 10)
mses = []
for order in orders:

    features = [np.ones_like(cars_train.weight)]
    for i in range(1, order+1):
        features.append(cars_train.weight.values**i)
    X = np.stack(features).T
    y = cars_train.mpg.values

    beta = np.linalg.inv(X.T @ X) @ X.T @ y

    features = [np.ones_like(cars_test.weight)]
    for i in range(1, order+1):
        features.append(cars_test.weight.values**i)
    X_test = np.stack(features).T

    test_mse = ((X_test @ beta - y_test)**2).mean()
    mses.append(test_mse)

plt.plot(orders, mses)
plt.xlabel("polynomial order")
plt.ylabel("test MSE")
plt.title("Polynomial out of sample error")


## In practice: linear regression using `sklearn`

Mostly we just use helper functions to do all of this:


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


Put together feature matrix and target vector


In [ ]:
X = cars[["cylinders", "displacement", "horsepower", "weight",
          "acceleration", "american", "european", "asian"]]
y = cars["mpg"]


Do train test split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


Fit using training set.


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)


Predict using the test features.


In [ ]:
yhat = model.predict(X_test)


Evaluate against the test target.


In [ ]:
mse = mean_squared_error(y_test, yhat)
print(mse)
rmse = np.sqrt(mse)
print(rmse)


^ The best yet. This model "wins" based on performance, but what influences mpg the most? Ah, if you care about that you're in the wrong place! What you want is inferential regression (and that's next).


## Jumping ahead: inference

Some assumptions that must hold in order to use linear regression for _inference_.


### Linearity: violated when you see nonlinear trends in your data / residuals


### Homoscedasticity: violated when the variance of your residuals isn't constant


### Normality: violated when the residuals are not normally distributed


### Multicollinearity: strictly violated when one feature is a linear combination of others, loosely violated when one feature is highly correlated with others
